<a href="https://colab.research.google.com/github/7201krap/PYTORCH_project/blob/main/all_seeds_batch_10_50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!tar -zxvf MNIST.tar.gz

root_dir = './'
torchvision.datasets.MNIST(root=root_dir,download=True)

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
mnist_trainset = torchvision.datasets.MNIST(root=root_dir, train=True, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

mnist_testset  = torchvision.datasets.MNIST(root=root_dir, 
                                train=False, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

train_dataloader = torch.utils.data.DataLoader(mnist_trainset, 
                                               batch_size=10, 
                                               shuffle=True)

test_dataloader  = torch.utils.data.DataLoader(mnist_testset, 
                                               batch_size=50, 
                                               shuffle=False)

In [5]:
# ************* modify this section for later use *************
# Define the model 
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        # modify this section for later use 
        self.linear_1 = torch.nn.Linear(784, 256)
        self.linear_2 = torch.nn.Linear(256, 10)
        self.sigmoid12  = torch.nn.Sigmoid()

        self.layer_activations = dict()

    def forward(self, x):

        # modify this section for later use 
        x = x.reshape(x.size(0), -1)
        x = self.linear_1(x)
        x = self.sigmoid12(x)
        pred = self.linear_2(x)
        return pred
# ************* modify this section for later use *************

In [6]:
def get_activation(model, layer_name):    
    def hook(module, input, output):
        model.layer_activations[layer_name] = output
    return hook

In [7]:
def sparsity_calculator(final_spareness):
    sparseness_list_avg = list()
    sparseness_list_std = list()
    for single_epoch_spareness in final_spareness:

        hidden_layer_activation_list = single_epoch_spareness
        hidden_layer_activation_list = torch.stack(hidden_layer_activation_list)
        layer_activations_list = torch.reshape(hidden_layer_activation_list, (10000, 256))

        layer_activations_list = torch.abs(layer_activations_list)  # modified 
        num_neurons = layer_activations_list.shape[1]
        population_sparseness = (np.sqrt(num_neurons) - (torch.sum(layer_activations_list, dim=1) / torch.sqrt(torch.sum(layer_activations_list ** 2, dim=1)))) / (np.sqrt(num_neurons) - 1)

        mean_sparseness_per_epoch = torch.mean(population_sparseness)
        std_sparseness_per_epoch  = torch.std(population_sparseness)

        sparseness_list_avg.append(mean_sparseness_per_epoch)
        sparseness_list_std.append(std_sparseness_per_epoch)

    return sparseness_list_avg, sparseness_list_std

In [8]:
def selectivity(hidden_layer_each_neuron):
    __selectivity__ = list()
    # I will now try to find the average of each class for each neuron.
    # check out the next cell 
    avg_activations = [dict() for x in range(256)]
    for i, neuron in enumerate(hidden_layer_each_neuron):
        for k, v in neuron.items():
            # v is the list of activations for hidden layer's neuron k 
            avg_activations[i][k] = sum(v) / float(len(v))

    # generate 256 lists to get only values in avg_activations
    only_activation_vals = [list() for x in range(256)]

    # get only values from avg_activations
    for i, avg_activation in enumerate(avg_activations):
        for value in avg_activation.values():
            only_activation_vals[i].append(value)

    for activation_val in only_activation_vals:
        # find u_max 
        u_max = np.max(activation_val)

        # find u_minus_max 
        u_minus_max = (np.sum(activation_val) - u_max) / 9

        # find selectivity 
        selectivity = (u_max - u_minus_max) / (u_max + u_minus_max)

        # append selectivity value to selectivity
        __selectivity__.append(selectivity)

    avg_selectivity = np.average(__selectivity__)
    std_selectivity = np.std(__selectivity__)
                                 
    return avg_selectivity, std_selectivity

In [9]:
def model_factory(optimizer_name, seed_num):
    '''
    optimizer_name : choose one of Adagrad, Adadelta, SGD, and Adam 

    '''
    my_model = Model()
    print("my_model:", my_model)
    my_model.to(device)

    # ************* modify this section for later use *************
    my_model.sigmoid12.register_forward_hook(get_activation(my_model, 's12'))
    # ************* modify this section for later use *************

    if optimizer_name == 'Adadelta':
        my_optimizer = torch.optim.Adadelta(my_model.parameters(), lr=1.0)

    elif optimizer_name == 'Adagrad':
        my_optimizer = torch.optim.Adagrad(my_model.parameters(), lr=0.1)

    elif optimizer_name == 'SGD':
        my_optimizer = torch.optim.SGD(my_model.parameters(), lr=0.1)

    elif optimizer_name == 'Adam':
        my_optimizer = torch.optim.Adam(my_model.parameters(), lr=0.001)

    else:
        print("ERROR")
    
    print("my_optimizer:", my_optimizer)
    test_acc, sparsity_avg, sparsity_std, selectivity_list_avg, selectivity_list_std = sparsity_selectivity_trainer(optimizer=my_optimizer, model=my_model)

    # ************* modify this section for later use *************
    file_saver = open(f"seed_{seed_num}_batchsize_10_50_{optimizer_name}.txt", "w")
    # ************* modify this section for later use *************
    file_saver.write(str(test_acc)+'\n'+str(sparsity_avg)+'\n'+str(sparsity_std)+'\n'+str(selectivity_list_avg)+'\n'+str(selectivity_list_std)+'\n\n')
    file_saver.close()

    if seed_num == 1:
    # ************* modify this section for later use *************
        if optimizer_name == 'Adadelta':
            !cp seed_1_batchsize_10_50_Adadelta.txt /content/drive/MyDrive
        
        elif optimizer_name == 'Adagrad':
            !cp seed_1_batchsize_10_50_Adagrad.txt /content/drive/MyDrive

        elif optimizer_name == 'SGD':
            !cp seed_1_batchsize_10_50_SGD.txt /content/drive/MyDrive

        elif optimizer_name == 'Adam':
            !cp seed_1_batchsize_10_50_Adam.txt /content/drive/MyDrive
        # ************* modify this section for later use *************
    
    elif seed_num == 100:
        # ************* modify this section for later use *************
        if optimizer_name == 'Adadelta':
            !cp seed_100_batchsize_10_50_Adadelta.txt /content/drive/MyDrive
        
        elif optimizer_name == 'Adagrad':
            !cp seed_100_batchsize_10_50_Adagrad.txt /content/drive/MyDrive

        elif optimizer_name == 'SGD':
            !cp seed_100_batchsize_10_50_SGD.txt /content/drive/MyDrive

        elif optimizer_name == 'Adam':
            !cp seed_100_batchsize_10_50_Adam.txt /content/drive/MyDrive
        # ************* modify this section for later use *************

    elif seed_num == 1234:
        # ************* modify this section for later use *************
        if optimizer_name == 'Adadelta':
            !cp seed_1234_batchsize_10_50_Adadelta.txt /content/drive/MyDrive
        
        elif optimizer_name == 'Adagrad':
            !cp seed_1234_batchsize_10_50_Adagrad.txt /content/drive/MyDrive

        elif optimizer_name == 'SGD':
            !cp seed_1234_batchsize_10_50_SGD.txt /content/drive/MyDrive

        elif optimizer_name == 'Adam':
            !cp seed_1234_batchsize_10_50_Adam.txt /content/drive/MyDrive
        # ************* modify this section for later use *************

    else:
        print("ERROR")

In [10]:
# ************* modify this section for later use *************
def avg_std_calculator(_hidden_layer_each_neuron_12):

    avg_selectivity12, std_selectivity12 = selectivity(_hidden_layer_each_neuron_12)

    final_selectivity_avg = (avg_selectivity12) / 1
    final_selecvitity_std = (std_selectivity12) / 1

    return final_selectivity_avg, final_selecvitity_std
# ************* modify this section for later use *************

In [11]:
no_epochs = 30
def sparsity_selectivity_trainer(optimizer, model):

    criterion = torch.nn.CrossEntropyLoss()
    train_loss = list()
    test_loss  = list()
    test_acc   = list()

    best_test_loss = 1

    selectivity_avg_list = list()
    selectivity_std_list = list()

    # ************* modify this section for later use *************
    final_spareness_12 = list()
    # ************* modify this section for later use *************

    for epoch in range(no_epochs):

        # ************* modify this section for later use *************
        hidden_layer_activation_list_12 = list()
        # ************* modify this section for later use *************

        # ************* modify this section for later use *************
        hidden_layer_each_neuron_12 = [{0:[], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[], 8:[], 9:[]} for x in range(256)]
        hidden_layer_each_neuron_12 = np.array(hidden_layer_each_neuron_12)
        # ************* modify this section for later use *************


        total_train_loss = 0
        total_test_loss = 0

        # training
        # set up training mode 
        model.train()

        for itr, (images, labels) in enumerate(train_dataloader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            pred = model(images)

            loss = criterion(pred, labels)
            total_train_loss += loss.item()

            loss.backward()
            optimizer.step()

        total_train_loss = total_train_loss / (itr + 1)
        train_loss.append(total_train_loss)

        # testing 
        # change to evaluation mode 
        model.eval()
        total = 0
        for itr, (images, labels) in enumerate(test_dataloader):
            images, labels = images.to(device), labels.to(device)

            pred = model(images)

            loss = criterion(pred, labels)
            total_test_loss += loss.item()

            # we now need softmax because we are testing.
            pred = torch.nn.functional.softmax(pred, dim=1)
            for i, p in enumerate(pred):
                if labels[i] == torch.max(p.data, 0)[1]:
                    total = total + 1

            # ***************** sparsity calculation ***************** #
            hidden_layer_activation_list_12.append(model.layer_activations['s12'])
            # ***************** sparsity calculation ***************** #

            # ************* modify this section for later use *************
            for activation, label in zip(model.layer_activations['s12'], labels):
                label = label.item()
                with torch.no_grad():
                    activation = activation.numpy()
                for i in range(256):    
                    hidden_layer_each_neuron_12[i][label].append(activation[i])
    
        selectivity_avg, selecvitity_std = avg_std_calculator(hidden_layer_each_neuron_12)
        # ************* modify this section for later use *************

        # this conains activations for all epochs 
        final_spareness_12.append(hidden_layer_activation_list_12)
        # ***************** sparsity calculation ***************** #
        
        selectivity_avg_list.append(selectivity_avg)
        selectivity_std_list.append(selecvitity_std)

        # caculate accuracy 
        accuracy = total / len(mnist_testset)

        # append accuracy here
        test_acc.append(accuracy)

        # append test loss here 
        total_test_loss = total_test_loss / (itr + 1)
        test_loss.append(total_test_loss)

        print('\nEpoch: {}/{}, Train Loss: {:.8f}, Test Loss: {:.8f}, Test Accuracy: {:.8f}'.format(epoch + 1, no_epochs, total_train_loss, total_test_loss, accuracy))

    # ***************** sparsity calculation ***************** #
    sparsity_list12_avg, sparsity_list12_std = sparsity_calculator(final_spareness_12)

    average_sparsity = list()
    std_sparsity = list()

    for i in range(no_epochs):
        average_sparsity.append( (sparsity_list12_avg[i].item()) / 1 )
        std_sparsity.append( (sparsity_list12_std[i].item()) / 1 )
    # ***************** sparsity calculation ***************** #

    return test_acc, average_sparsity, std_sparsity, selectivity_avg_list, selectivity_std_list

In [12]:
# Seed 1

torch.manual_seed(1)
np.random.seed(1)

model_factory('Adadelta',1)
model_factory('Adagrad',1)
model_factory('SGD',1)
model_factory('Adam',1)

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
)
my_optimizer: Adadelta (
Parameter Group 0
    eps: 1e-06
    lr: 1.0
    rho: 0.9
    weight_decay: 0
)

Epoch: 1/30, Train Loss: 0.29458568, Test Loss: 0.16176601, Test Accuracy: 0.95170000

Epoch: 2/30, Train Loss: 0.13846570, Test Loss: 0.11701595, Test Accuracy: 0.96570000

Epoch: 3/30, Train Loss: 0.09929245, Test Loss: 0.09966182, Test Accuracy: 0.96950000

Epoch: 4/30, Train Loss: 0.07714660, Test Loss: 0.08492453, Test Accuracy: 0.97550000

Epoch: 5/30, Train Loss: 0.06368094, Test Loss: 0.08354693, Test Accuracy: 0.97550000

Epoch: 6/30, Train Loss: 0.05350632, Test Loss: 0.07365136, Test Accuracy: 0.97900000

Epoch: 7/30, Train Loss: 0.04615458, Test Loss: 0.07593191, Test Accuracy: 0.97840000

Epoch: 8/30, Train Loss: 0.03870709, Test Loss: 0.07615420, Test Accuracy: 0.97770000

Epoch: 9/30, Train Lo

In [13]:
# Seed 100

torch.manual_seed(100)
np.random.seed(100)

model_factory('Adadelta',100)
model_factory('Adagrad',100)
model_factory('SGD',100)
model_factory('Adam',100)

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
)
my_optimizer: Adadelta (
Parameter Group 0
    eps: 1e-06
    lr: 1.0
    rho: 0.9
    weight_decay: 0
)

Epoch: 1/30, Train Loss: 0.29700167, Test Loss: 0.15857965, Test Accuracy: 0.95170000

Epoch: 2/30, Train Loss: 0.14180494, Test Loss: 0.11529885, Test Accuracy: 0.96360000

Epoch: 3/30, Train Loss: 0.09991995, Test Loss: 0.09634405, Test Accuracy: 0.97220000

Epoch: 4/30, Train Loss: 0.07738981, Test Loss: 0.08497894, Test Accuracy: 0.97480000

Epoch: 5/30, Train Loss: 0.06347882, Test Loss: 0.07923635, Test Accuracy: 0.97640000

Epoch: 6/30, Train Loss: 0.05306248, Test Loss: 0.07337166, Test Accuracy: 0.97790000

Epoch: 7/30, Train Loss: 0.04475503, Test Loss: 0.07204167, Test Accuracy: 0.97830000

Epoch: 8/30, Train Loss: 0.03874095, Test Loss: 0.06436910, Test Accuracy: 0.97950000

Epoch: 9/30, Train Lo

In [14]:
# Seed 1234

torch.manual_seed(1234)
np.random.seed(1234)

model_factory('Adadelta',1234)
model_factory('Adagrad',1234)
model_factory('SGD',1234)
model_factory('Adam',1234)

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
)
my_optimizer: Adadelta (
Parameter Group 0
    eps: 1e-06
    lr: 1.0
    rho: 0.9
    weight_decay: 0
)

Epoch: 1/30, Train Loss: 0.29697952, Test Loss: 0.17111190, Test Accuracy: 0.94820000

Epoch: 2/30, Train Loss: 0.14156637, Test Loss: 0.11867337, Test Accuracy: 0.96360000

Epoch: 3/30, Train Loss: 0.09923429, Test Loss: 0.09834795, Test Accuracy: 0.97060000

Epoch: 4/30, Train Loss: 0.07722462, Test Loss: 0.08848850, Test Accuracy: 0.97250000

Epoch: 5/30, Train Loss: 0.06309570, Test Loss: 0.07919768, Test Accuracy: 0.97560000

Epoch: 6/30, Train Loss: 0.05276615, Test Loss: 0.07398020, Test Accuracy: 0.97710000

Epoch: 7/30, Train Loss: 0.04432759, Test Loss: 0.07586859, Test Accuracy: 0.97770000

Epoch: 8/30, Train Loss: 0.03784261, Test Loss: 0.07055158, Test Accuracy: 0.97940000

Epoch: 9/30, Train Lo